In [1]:
import os
import numpy as np
import pandas as pd
import rasterio
import tensorflow as tf
import sys
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Conv2D, UpSampling2D, concatenate, Input, BatchNormalization, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import rasterio
import re


# Preprocessing & Loading

In [2]:

base_path = './testset'
pattern = re.compile(r'smalldata_(\d+)_(\d+)')
indices = ['ExG', 'ExR', 'PRI', 'MGRVI', 'SAVI', 'MSAVI', 'EVI', 'REIP', 'NDVI', 'GNDVI', 'CI', 'OSAVI', 'TVI', 'MCARI', 'TCARI']

def load_tif(file_path):
    # Function to load .tif file and return as a numpy array using rasterio
    with rasterio.open(file_path) as src:
        return src.read(1)  # Read the first band

def read_X(dir=base_path, indices=indices):
    images = []
    labels = []
    for root, dirs, files in os.walk(dir):
        for dir_name in dirs:
            match = pattern.match(dir_name)
            if match:
                group_number = match.group(1)
                sub_group_number = match.group(2)
                dir_path = os.path.join(root, dir_name)
                channels = []
                skip_directory = False
                for file_name in os.listdir(dir_path):
                    if file_name.endswith('.tif'):
                        for index in indices:
                            if file_name.startswith(index):
                                file_path = os.path.join(dir_path, file_name)
                                data = load_tif(file_path)
                                if np.isnan(data).any():
                                    print(f"Skipping directory due to NaN values in: {file_path}")
                                    skip_directory = True
                                    break
                                channels.append(data)
                        if skip_directory:
                            break
                    if file_name.startswith("label_matrix"):
                        file_path = os.path.join(dir_path, file_name)
                        label_matrix = pd.read_csv(file_path, header=None).values
                if not skip_directory:
                    images.append(channels)
                    labels.append(label_matrix)
    return np.array(images), np.array(labels)

# # Example usage
# images, labels = read_X()
# print(images.shape, labels.shape)


In [3]:
X,y=read_X()

In [5]:
import numpy as np

def convert_to_one_hot(y):
    # Get the shape of the input array
    n, h, w = y.shape
    
    # Initialize an all-zero array with shape (n, h, w, 4)
    y_one_hot = np.zeros((n, h, w, 4), dtype=int)
    
    # Use advanced indexing to convert the original array values to one-hot encoding
    for i in range(4):
        y_one_hot[..., i] = (y == i)
    
    return y_one_hot


def get_predicted_labels(predictions):
    """
    Convert the predicted probability array to a label array.
    
    Parameters:
    predictions: A predicted probability array with shape (n, 512, 512, 4)
    
    Returns:
    A label array with shape (n, 512, 512), where each point represents its most probable class
    """
    predicted_labels = np.argmax(predictions, axis=-1)
    return predicted_labels


def one_hot_to_labels(y_one_hot):
    """
    Convert a one-hot encoded array back to a label array.
    
    Parameters:
    y_one_hot: A one-hot encoded array with shape (n, 512, 512, 4)
    
    Returns:
    A label array with shape (n, 512, 512)
    """
    # Use np.argmax to find the index of the maximum value in the fourth dimension
    y_labels = np.argmax(y_one_hot, axis=-1)
    
    return y_labels


In [6]:
# Normalize input data to [0, 1]
X = X / np.max(X)

# Transpose the dimensions of X to (0, 2, 3, 1)
X = X.transpose((0, 2, 3, 1))

# Convert y to one-hot encoding
y_one_hot = convert_to_one_hot(y)


In [14]:
y_one_hot.shape

(4, 512, 512, 4)

In [7]:
import tensorflow as tf

def preprocess_image(image, label):
    # Do not resize, keep the original dimensions
    return image, label

def load_dataset(images, labels, batch_size=4):
    # Create a TensorFlow dataset from the images and labels
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    
    # Apply the preprocess_image function to each element in the dataset
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    # Batch the dataset and prefetch for better performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

# Load the training dataset
train_dataset = load_dataset(X, y_one_hot)



# Using ResNet50 from Keras


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, Dropout
from tensorflow.keras.models import Model

# Define a custom ResizeLayer
class ResizeLayer(tf.keras.layers.Layer):
    def __init__(self, target_height, target_width):
        super(ResizeLayer, self).__init__()
        self.target_height = target_height
        self.target_width = target_width

    def call(self, inputs):
        return tf.image.resize(inputs, (self.target_height, self.target_width))

# Define the input tensor with shape (512, 512, 15)
input_tensor = Input(shape=(512, 512, 15))

# Add a convolutional layer to convert the input to a 3-channel input suitable for ResNet50
x = Conv2D(3, (1, 1), padding='same', activation='relu')(input_tensor)
print(x.shape)  # 512

# Use the pre-trained ResNet50 model, excluding the top classification layer
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(512, 512, 3))

# Connect the custom input layer to the base model
x = base_model(x)

# Use convolutional layers to maintain the spatial dimensions
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
print(x.shape)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
print(x.shape)

# Add the final convolutional layer
x = Conv2D(4, (1, 1), padding='same')(x)
print(x.shape)

# Use the custom resize layer to resize the output to (512, 512)
x = ResizeLayer(512, 512)(x)
print(x.shape)

# Apply the Softmax activation function to ensure the output represents a probability distribution
predictions = tf.keras.layers.Softmax(axis=-1)(x)

# Define the model with the input tensor and the predictions
model = Model(inputs=input_tensor, outputs=predictions)

# Freeze the convolutional layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with RMSprop optimizer and categorical cross-entropy loss
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Uncomment the lines below to print the predictions shape and verify the sum of probabilities
# print(predictions.shape)
# print(predictions)
# print(np.sum(predictions[0, :, :, :], axis=-1))


(None, 512, 512, 3)
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step
(None, 16, 16, 512)
(None, 16, 16, 512)
(None, 16, 16, 4)
(None, 512, 512, 4)


# ResNet-11

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Define a basic residual block
def residual_block(x, filters, kernel_size=3, stride=1):
    y = Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    y = BatchNormalization()(y)
    y = ReLU()(y)
    y = Conv2D(filters, kernel_size, strides=1, padding='same')(y)
    y = BatchNormalization()(y)

    if stride != 1 or x.shape[-1] != filters:
        x = Conv2D(filters, 1, strides=stride, padding='same')(x)
        x = BatchNormalization()(x)

    out = Add()([x, y])
    out = ReLU()(out)
    return out

# Define the input tensor with shape (512, 512, 15)
input_tensor = Input(shape=(512, 512, 15))

# Initial convolutional layer to convert input to a 3-channel image
x = Conv2D(64, (3, 3), padding='same', activation='relu')(input_tensor)
x = BatchNormalization()(x)

# Stack of residual blocks
x = residual_block(x, 64)
x = residual_block(x, 64)
x = residual_block(x, 128, stride=2)  # downsample
x = residual_block(x, 128)
x = residual_block(x, 256, stride=2)  # downsample
x = residual_block(x, 256)
x = residual_block(x, 512, stride=2)  # downsample
x = residual_block(x, 512)

# Final convolutional layers to maintain the spatial dimensions
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)

# Add the final convolutional layer
x = Conv2D(4, (1, 1), padding='same')(x)

# Use the custom resize layer to resize the output to (512, 512)
class ResizeLayer(tf.keras.layers.Layer):
    def __init__(self, target_height, target_width):
        super(ResizeLayer, self).__init__()
        self.target_height = target_height
        self.target_width = target_width

    def call(self, inputs):
        return tf.image.resize(inputs, (self.target_height, self.target_width))

x = ResizeLayer(512, 512)(x)

# Apply the Softmax activation function to ensure the output represents a probability distribution
predictions = tf.keras.layers.Softmax(axis=-1)(x)

# Define the model with the input tensor and the predictions
model = Model(inputs=input_tensor, outputs=predictions)

# Compile the model with RMSprop optimizer and categorical cross-entropy loss
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 15)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 512, 512,  │      8,704 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512, 512,  │        256 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 512, 512,  │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512, 512,  │        256 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 512, 512,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 512, 512,  │     36,928 │ re_lu[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512, 512,  │        256 │ conv2d_6[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 512, 512,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 512, 512,  │          0 │ add[0][0]         │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 512, 512,  │     36,928 │ re_lu_1[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512, 512,  │        256 │ conv2d_7[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 512, 512,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 512, 512,  │     36,928 │ re_lu_2[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512, 512,  │        256 │ conv2d_8[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 512, 512,  │          0 │ re_lu_1[0][0],    │
│                     │ 64)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 512, 512,  │          0 │ add_1[0][0]     

 Total params: 15,911,812 (60.70 MB)

 Trainable params: 15,902,212 (60.66 MB)

 Non-trainable params: 9,600 (37.50 KB)

# Training and Saving as .h5 file

In [16]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_loss') <= 0.1099 and logs.get('loss') <= 0.1099:
            print('\n\n Reached The Destination!')
            self.model.stop_training = True

callbacks = myCallback()
history = model.fit(
    train_dataset,
    epochs=10
    # callbacks=[callbacks]
)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.2016 - loss: 1.4640
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step - accuracy: 0.8717 - loss: 87.7566
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step - accuracy: 0.6184 - loss: 0.8022
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step - accuracy: 0.8717 - loss: 0.4787
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.8717 - loss: 0.4678
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 18s 18s/step - accuracy: 0.8717 - loss: 0.4736
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step - accuracy: 0.8717 - loss: 0.4318
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.8717 - loss: 0.3926
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - accuracy: 0.8717 - loss: 0.3779
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step - accuracy: 0.8717 - loss: 0.3728


In [17]:
model_path='./model_7_8.h5'

model.save(model_path)
print(model_path)


if os.path.exists(model_path):
    print(f"Model saved successfully at {model_path}")
else:
    print(f"Model not found at {model_path}")

./inceptionv3_fcn_model_23_5.h5
Model saved successfully at ./inceptionv3_fcn_model_23_5.h5


# make pred and calculate metrics

In [18]:
from keras.models import load_model
import tensorflow as tf

# Define custom layer without 'trainable' argument
class ResizeLayer(tf.keras.layers.Layer):
    def __init__(self, target_height, target_width, **kwargs):
        super(ResizeLayer, self).__init__(**kwargs)
        self.target_height = target_height
        self.target_width = target_width

    def call(self, inputs):
        return tf.image.resize(inputs, [self.target_height, self.target_width])

# Add custom layer to the custom_objects dictionary
custom_objects = {'ResizeLayer': ResizeLayer}

# Load the model with the custom objects
with tf.keras.utils.custom_object_scope(custom_objects):
    model = load_model('./model_7_8.h5', compile=False)


In [19]:
def make_pred(dir):
    # Read the test data from the specified directory
    X_test, y_test = read_X(dir=dir)
    
    # Print the shape and data type of the test data
    print(f"Shape of X_test: {X_test.shape}")
    print(f"Data type of X_test: {X_test.dtype}")
    
    # Normalize the test data to the range [0, 1]
    X_test = X_test / np.max(X_test)
    
    # Transpose the dimensions of X_test to (0, 2, 3, 1)
    X_test = X_test.transpose((0, 2, 3, 1))
    
    # Make predictions using the model
    predictions = model.predict(X_test)
    
    # Uncomment the following line to print the predictions
    # print(predictions)
    
    # Convert y_test to one-hot encoding
    y_test_one_hot = convert_to_one_hot(y_test)
    
    return y_test_one_hot, predictions


In [21]:
y_test,y_pred= make_pred(dir='./testset')


Shape of X_test: (4, 15, 512, 512)
Data type of X_test: float32
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


In [22]:
print(y_pred)

[[[[2.9391202e-01 2.5852394e-01 2.4474546e-01 2.0281853e-01]
   [2.9391202e-01 2.5852394e-01 2.4474546e-01 2.0281853e-01]
   [2.9391202e-01 2.5852394e-01 2.4474546e-01 2.0281853e-01]
   ...
   [2.9070356e-01 2.5711584e-01 2.4429673e-01 2.0788372e-01]
   [2.9070356e-01 2.5711584e-01 2.4429673e-01 2.0788372e-01]
   [2.9070356e-01 2.5711584e-01 2.4429673e-01 2.0788372e-01]]

  [[2.9391202e-01 2.5852394e-01 2.4474546e-01 2.0281853e-01]
   [2.9391202e-01 2.5852394e-01 2.4474546e-01 2.0281853e-01]
   [2.9391202e-01 2.5852394e-01 2.4474546e-01 2.0281853e-01]
   ...
   [2.9070356e-01 2.5711584e-01 2.4429673e-01 2.0788372e-01]
   [2.9070356e-01 2.5711584e-01 2.4429673e-01 2.0788372e-01]
   [2.9070356e-01 2.5711584e-01 2.4429673e-01 2.0788372e-01]]

  [[2.9391202e-01 2.5852394e-01 2.4474546e-01 2.0281853e-01]
   [2.9391202e-01 2.5852394e-01 2.4474546e-01 2.0281853e-01]
   [2.9391202e-01 2.5852394e-01 2.4474546e-01 2.0281853e-01]
   ...
   [2.9070356e-01 2.5711584e-01 2.4429673e-01 2.0788372e-01]

In [23]:
def calculate_accuracy(y_true, y_pred, num_classes=4):
    """
    Calculate classification accuracy.
    
    :param y_true: Actual labels, shape (batch_size, height, width, num_classes)
    :param y_pred: Predicted labels, shape (batch_size, height, width, num_classes)
    :param num_classes: Number of classes
    :return: Classification accuracy
    """
    # Convert one-hot encoded labels to class indices
    y_true_class = np.argmax(y_true, axis=-1)
    y_pred_class = np.argmax(y_pred, axis=-1)
    
    # Calculate accuracy
    correct_predictions = np.sum(y_true_class == y_pred_class)
    total_predictions = y_true_class.size
    
    accuracy = correct_predictions / total_predictions
    return accuracy

# Call the function to calculate accuracy
calculate_accuracy(y_test, y_pred, num_classes=4)


0.8716917037963867

Are all sums of the four channels equal to 1? True
